# Projeto 1 - módulo 6

## Precificação dinâmica - e-commerce

### Mercari Price Suggestion Challenge - Kaggle

Mercari é um site de revenda de produtos online. Uma dos desafios desse tipo de plataforma é auxiliar o usuário, muitas vezes com pouco conhecimento de vendas, a determinar um preço para os seus produtos de modo a maximizar as chances de venda.

### Sobre este projeto

O presente projeto tem o objetivo de desenvolver um algoritmo que identifique produtos já vendidos similares e sugira ao usuário um preço ótimo para novos produtos cadastrados.


### Preparação do ambiente

Para este projeto, acesse o link https://www.kaggle.com/competitions/mercari-price-suggestion-challenge/overview 


In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers
from tensorflow.keras.models import Model

!pip install tensorflow_addons
import tensorflow_addons as tfa
from sklearn import metrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
base_train = pd.read_csv('/content/drive/MyDrive/Blue Edtech/data/processed/dados_treino.csv') 
base_test = pd.read_csv('/content/drive/MyDrive/Blue Edtech/data/processed/dados_teste.csv')
base_val = pd.read_csv('/content/drive/MyDrive/Blue Edtech/data/processed/dados_validacao.csv') 

In [4]:
# preenchendo dados NAN

base_train['item_description'] = base_train['item_description'].fillna("No description")
base_train['name'] = base_train['name'].fillna(" ")

base_test['item_description'] = base_test['item_description'].fillna("No description")
base_test['name'] = base_test['name'].fillna(" ")

base_val['item_description'] = base_val['item_description'].fillna("No description")
base_val['name'] = base_val['name'].fillna(" ")

In [23]:
# unindo os atributos textuais buscando fortalecer a relação semântica entre eles

base_train['name_brand_description'] = base_train['name'] + ' ' + base_train['brand_name'] + ' ' + base_train['item_description']
base_test['name_brand_description'] = base_test['name'] + ' ' + base_test['brand_name'] + ' ' + base_test['item_description']
base_val['name_brand_description'] = base_val['name'] + ' ' + base_val['brand_name'] + ' ' + base_val['item_description']

In [25]:
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [7]:
# removendo stopwords
# stop_words = stopwords.words('english')
# base_train['name_brand_description'] = base_train['name_brand_description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
# base_test['name_brand_description'] = base_test['name_brand_description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
# base_val['name_brand_description'] = base_val['name_brand_description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [8]:
# tokenizando
# tt = TweetTokenizer()
# base_train['name_brand_description'] = base_train['name_brand_description'].apply(tt.tokenize)
# base_test['name_brand_description'] = base_test['name_brand_description'].apply(tt.tokenize)
# base_val['name_brand_description'] = base_val['name_brand_description'].apply(tt.tokenize)

In [ ]:
# Lemmatise
# lmtzr = WordNetLemmatizer()
# base_train['name_brand_description'] = base_train['name_brand_description'].apply(lambda x:[lmtzr.lemmatize(word) for word in x])
# base_test['name_brand_description'] = base_test['name_brand_description'].apply(lambda x:[lmtzr.lemmatize(word) for word in x])
# base_val['name_brand_description'] = base_val['name_brand_description'].apply(lambda x:[lmtzr.lemmatize(word) for word in x])

In [9]:
# Stemming
# stemmer = SnowballStemmer("english")
# base_train['name_brand_description'] = base_train['name_brand_description'].apply(lambda x: [stemmer.stem(word) for word in x])
# base_test['name_brand_description'] = base_test['name_brand_description'].apply(lambda x: [stemmer.stem(word) for word in x])
# base_val['name_brand_description'] = base_val['name_brand_description'].apply(lambda x: [stemmer.stem(word) for word in x])

In [26]:
# removendo pontuação

base_train['name_brand_description'] = base_train['name_brand_description'].apply(lambda x:[re.sub(r'[^\w\s]','', str(x))])
base_test['name_brand_description'] = base_test['name_brand_description'].apply(lambda x:[re.sub(r'[^\w\s]','', str(x))])
base_val['name_brand_description'] = base_val['name_brand_description'].apply(lambda x:[re.sub(r'[^\w\s]','', str(x))])

In [ ]:
# salvando dataset tratado com stemming

# base_train.to_csv("stemming_train.csv")
# base_test.to_csv("stemming_test.csv")
# base_val.to_csv('stemming_val.csv')

In [ ]:
# salvando dataset tratado com lemmatise

# base_train.to_csv("lemma_train.csv")
# base_test.to_csv("lemma_test.csv")
# base_val.to_csv('lemma_val.csv')

In [27]:
# função de tokenização e preenchimento de comprimento

def text_vectorizer(feature):

  # TOKENIZAÇÃO
  tk = Tokenizer()

  # FIT ON TRAIN 
  tk.fit_on_texts(base_train[feature].apply(str))

  # TOKENIZANDO O DATASET DE TREINO
  tk_train = tk.texts_to_sequences(base_train[feature].apply(str))

  # TOKENIZANDO O DATASET DE TESTE
  tk_test = tk.texts_to_sequences(base_test[feature].apply(str))

  # TOKENIZANDO O DATASET DE VALIDAÇÃO
  tk_val = tk.texts_to_sequences(base_val[feature].apply(str))

  # COMPUTANDO O COMPRIMENTO MÁXIMO
  max_length = base_train[feature].apply(lambda x :len(str(x).split())).max()

  # COMPUTANDO O TAMANHO DO VOCABULÁRIO
  vocab_size = len(tk.word_index) + 1

  # PADDING A SEQUENCIA DE TREINO
  train_pad= pad_sequences(tk_train,padding="post",maxlen = max_length)

  # PADDING A SEQUENCIA DE TESTE
  test_pad = pad_sequences(tk_test,padding = "post", maxlen = max_length)

  # PADDING A SEQUENCIA DE TESTE
  val_pad = pad_sequences(tk_val,padding = "post", maxlen = max_length)

  # RETURNANDO O COMPRIMENTO MÁXIMO, SEQUÊNCIA PADDED DE TRAINO  , SEQUENCIA PADDED DE VALIDAÇÃO 
  return max_length, vocab_size, train_pad , test_pad, val_pad

In [28]:
# rodando a função text_vectorizer para todos os atributos

max_length_name_brand_description, vocab_size_name_brand_description, train_name_brand_description_pad, test_name_brand_description_pad, val_name_brand_description_pad = text_vectorizer('name_brand_description')  
max_length_category_1, vocab_size_category_1, train_category_1_pad , test_category_1_pad, val_category_1_pad = text_vectorizer('category_1')
max_length_category_2, vocab_size_category_2, train_category_2_pad , test_category_2_pad, val_category_2_pad = text_vectorizer('category_2')
max_length_category_3, vocab_size_category_3, train_category_3_pad , test_category_3_pad, val_category_3_pad = text_vectorizer('category_3')

In [14]:
# separando treino e teste do atributo shipping

train_shipping = base_train.shipping
test_shipping = base_test.shipping
val_shipping = base_val.shipping

# separando treino e teste do atributo item_condition_id

train_item_cond = base_train.item_condition_id
test_item_cond = base_test.item_condition_id
val_item_cond = base_val.item_condition_id


In [31]:
# armazenando os dados em uma lista

x_train = [train_item_cond,train_shipping,train_category_1_pad,train_category_2_pad,train_category_3_pad,train_name_brand_description_pad]
x_test = [test_item_cond,test_shipping,test_category_1_pad, test_category_2_pad, test_category_3_pad,test_name_brand_description_pad]
x_val = [val_item_cond,val_shipping,val_category_1_pad, val_category_2_pad, val_category_3_pad,val_name_brand_description_pad]

In [32]:
# convertendo formato do atributo price para log

base_train['log_price'] = np.log(base_train['price'])
base_test['log_price'] = np.log(base_test['price'])

y_train = base_train.log_price
y_test = base_test.log_price

In [33]:
# arquitetura do deep learning

tf.keras.backend.clear_session()
# ITEM CONDITION ID
inp1 = layers.Input(shape=(1)) # INPUT 1 
d1 = layers.Dense(10,activation="relu")(inp1) # DENSE LAYER 1
# SHIPPING 
inp2 = layers.Input(shape=(1)) # INPUT 2 
d2 = layers.Dense(10,activation="relu")(inp2) # DENSE LAYER 2
# CATEGORY_1
inp3 = layers.Input(shape = (3)) # INPUT 4
emb3 = layers.Embedding(vocab_size_category_1, 16, input_length= 3)(inp3) # EMBEDDING 4
flat3 = layers.Flatten()(emb3) # FLATTEN 
# CATEGORY_2
inp4 = layers.Input(shape = (5)) # INPUT 5
emb4 = layers.Embedding(vocab_size_category_2 , 16, input_length= 5)(inp4) # EMBEDDING 5
flat4 = layers.Flatten()(emb4) # FLATTEN
# CATEGORY_3
inp5= layers.Input(shape = (7)) # INPUT 6 
emb5 = layers.Embedding(vocab_size_category_3, 40, input_length= 7)(inp5) # EMBEDDING 6
flat5 = layers.Flatten()(emb5) # FLATTEN
# ITEM NAME_BRAND_DESCRIPTION
inp6= layers.Input(shape = (220)) # INPUT 8
emb6 = layers.Embedding(vocab_size_name_brand_description , 20 , input_length= 220 )(inp6) # EMBEDDING 8
lstm6 = layers.GRU(254,return_sequences=True)(emb6) # GRU
flat6 = layers.Flatten()(lstm6) # FLATTEN
# CONCATENAÇÃO
concat = layers.Concatenate()([d1,d2,flat3,flat4,flat5,flat6])
# DENSE LAYERS
dense1 = layers.Dense(512,activation="relu")(concat)
# DROPOUT LAYER
drop1 = layers.Dropout(0.1)(dense1)
# DENSE LAYER
dense2 = layers.Dense(256,activation="relu")(drop1)
# DROPOUT LAYER
drop2 = layers.Dropout(0.1)(dense2)
# DENSE LAYER
dense3 = layers.Dense(128,activation="relu")(drop2)
# BATCHNORM LAYER
bn2  = layers.BatchNormalization()(dense3)
# DENSE LAYER
dense4 = layers.Dense(1,activation="linear")(bn2)
# MODEL
model =  Model(inputs= [inp1,inp2,inp3,inp4,inp5,inp6],outputs=dense4)

# SCHEDULE
def shedule(epoch,lr):
    if epoch<=2:
        return lr
    else:
        return lr*0.1
# CALLBACKS
lr = tf.keras.callbacks.LearningRateScheduler(shedule,verbose=1)
save = tf.keras.callbacks.ModelCheckpoint("content/drive/MyDrive/Blue Edtech/notebooks",monitor="val_root_mean_squared_error",mode="min",save_best_only=True, save_weights_only=True,verbose=1)
earlystop = tf.keras.callbacks.EarlyStopping(monitor="val_root_mean_squared_error",min_delta= 0.01, patience=2,mode="min" )

model.compile(optimizer="adam",loss="mse",metrics=  [tf.keras.losses.MeanAbsoluteError(), tfa.metrics.r_square.RSquare(), tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.mean_absolute_percentage_error , tf.keras.metrics.mean_squared_logarithmic_error ])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 220)]        0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 3)]          0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 5)]          0           []                               
                                                                                                  
 input_5 (InputLayer)           [(None, 7)]          0           []                               
                                                                                              

In [34]:
# FITTING THE MODEL
history = model.fit(x_train, y_train, validation_data= (x_test, y_test), epochs=10, batch_size = 1024, callbacks=[save,lr,earlystop])


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/10
912/912 [==============================] - ETA: 0s - loss: 0.7248 - mean_absolute_error: 0.5517 - r_square: -0.1628 - root_mean_squared_error: 0.8513 - mean_absolute_percentage_error: 20.0323 - mean_squared_logarithmic_error: 0.0882
Epoch 1: val_root_mean_squared_error improved from inf to 0.54953, saving model to content/drive/MyDrive/Blue Edtech/notebooks
912/912 [==============================] - 215s 233ms/step - loss: 0.7248 - mean_absolute_error: 0.5517 - r_square: -0.1628 - root_mean_squared_error: 0.8513 - mean_absolute_percentage_error: 20.0323 - mean_squared_logarithmic_error: 0.0882 - val_loss: 0.3020 - val_mean_absolute_error: 0.4214 - val_r_square: 0.5144 - val_root_mean_squared_error: 0.5495 - val_mean_absolute_percentage_error: 14.5197 - val_mean_squared_logarithmic_error: 0.0206 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/10

In [35]:
# CRIANDO A FUNÇÃO DE CÁLCULO DAS MÉTRICAS

def print_avaliacao(obs, pred):
    print('R² = %.3f' % metrics.r2_score(obs, pred))
    print('MAPE = %.3f %%' % (100 * metrics.mean_absolute_percentage_error(obs, pred)))
    print('MAE = U$S %.2f' % (metrics.mean_absolute_error(obs, pred)))
    print('RMSE = U$S %.2f' % metrics.mean_squared_error(obs, pred)**0.5)
    print('RMSLE = %.4f' % metrics.mean_squared_log_error(obs, pred,squared=False))

In [36]:
# Predição e avaliação da amostra teste

y_pred = np.exp(model.predict(x_test))

12502/12502 [==============================] - 84s 7ms/step


In [37]:
print_avaliacao(base_test.price,y_pred)

R² = 0.554
MAPE = 37.507 %
MAE = U$S 9.81
RMSE = U$S 25.85
RMSLE = 0.4420


In [38]:
# Predição e avaliação da amostra de validação

y_pred_val = np.exp(model.predict(x_val))
print_avaliacao(base_val.price,y_pred_val)

4631/4631 [==============================] - 31s 7ms/step
R² = 0.526
MAPE = 37.513 %
MAE = U$S 9.75
RMSE = U$S 26.39
RMSLE = 0.4426


Avaliação do BIAS do modelo

In [ ]:
# criando coluna para os valores preditos

base_test['y_pred'] = y_pred
base_val['y_pred_val'] = y_pred_val

In [ ]:
# calculando o BIAS

bias = (base_test['y_pred'] - base_test['price']).mean()
print(bias)

In [ ]:
# visualizando tendência do modelo

import seaborn as sns

hist = (base_test['y_pred'] - base_test['price'])
sns.histplot(np.log(hist))

In [ ]:
# calculando normalidade da distribuição

from scipy import stats

k, p = stats.normaltest(hist)
p

Como valor de p é 0 isto significa dizer que a distribuição é normal e portanto o viés não é estatisticamente significante.

In [ ]:
# base_test.to_csv("resultados_test.csv")
# base_val.to_csv('resultados_val.csv')

### Testando o uso do modelo na prática

In [ ]:
# Venda seu produto aqui no ValorA.I.!

nome = input('Digite o nome do produto: \n')
marca = input('Digite a marca do produto: \n')
descricao = input('Digite a descrição do produto: \n')
categoria_1 = input('Digite a categoria 1 do produto: \n')
categoria_2 = input('Digite a categoria 2 do produto: \n')
categoria_3 = input('Digite a categoria 3 do produto: \n')
anuncio = pd.DataFrame({'nome': [nome],'marca' : [marca],'descricao': [descricao],'categoria_1': [categoria_1],'categoria_2': [categoria_2], 'categoria_3': [categoria_3] })

In [ ]:
anuncio['name_brand_description'] = anuncio['nome'] + ' ' + anuncio['marca'] + ' ' + anuncio['descricao']

In [ ]:
# tratando as entradas do anúncio

#vetorizando name_brand_description
tk_anuncio = tk__name_brand_description.texts_to_sequences(anuncio['name_brand_description'].apply(str))
anuncio_name_brand_description_pad = pad_sequences(tk_anuncio,padding="post",maxlen = 254)

#vetorizando categoria_1
tk_anuncio = tk_category_1.texts_to_sequences(anuncio['categoria_1'].apply(str))
anuncio_category_1_pad = pad_sequences(tk_anuncio,padding="post",maxlen = 3)

#vetorizando categoria_2
tk_anuncio = tk_category_2.texts_to_sequences(anuncio['categoria_2'].apply(str))
anuncio_category_2_pad = pad_sequences(tk_anuncio,padding="post",maxlen = 5)

#vetorizando categoria_3
tk_anuncio = tk_category_3.texts_to_sequences(anuncio['categoria_3'].apply(str))
anuncio_category_3_pad = pad_sequences(tk_anuncio,padding="post",maxlen = 7)

In [ ]:
# atribunindo valor para atributos categoricos

anuncio_item_cond = pd.Series([4])
anuncio_shipping = pd.Series([0])

In [ ]:
x_anuncio = [anuncio_item_cond,anuncio_shipping,anuncio_category_1_pad,anuncio_category_2_pad,anuncio_category_3_pad,anuncio_name_brand_description_pad]

In [ ]:
y_pred_anuncio = np.exp(model.predict(x_anuncio))
print(f' Sugerimos um valor entre U${np.round(y_pred_anuncio*0.8,2)} e U${np.round(y_pred_anuncio*1.2,2)}, um preço ideal seria U${np.round(y_pred_anuncio,2)}')

1/1 [==============================] - 0s 22ms/step
 Sugerimos um valor entre U$[[35.33]] e U$[[53.]], um preço ideal seria U$[[44.16]]


Visualização da arquitetura da rede

### Referência

https://github.com/pushapgandhi/Mercari_Price_Prediction/blob/main/Deep_Learning_Model.ipynb

https://towardsdatascience.com/mercari-price-prediction-challenge-3a8ea00a7d33

